### Import Libraries and Data

In [1]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference
import string
from openpyxl.styles import numbers, Alignment, Border, Side
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter

In [2]:
df = pd.read_csv('data\AdvWorksData.csv', encoding="ISO-8859-1")

In [3]:
df = df[['productcategory', 'Country', 'Sales']]
df

,productcategory,Country,Sales
0,Clothing,United Kingdom,68.786592
1,Accessories,United Kingdom,90.000000
2,Clothing,United Kingdom,182.352000
3,Clothing,United Kingdom,317.596400
4,Clothing,United Kingdom,159.558000
...,...,...,...
60875,Clothing,United States,335.952000
60876,Clothing,United States,251.964000
60877,Clothing,United States,125.982000
60878,Clothing,United States,209.970000


### Create Pivot Table

In [4]:
report_table = df.pivot_table(index='productcategory',
                                      columns='Country',
                                      values='Sales',
                                      aggfunc='sum').round(0)
report_table

Country,Australia,Canada,France,Germany,United Kingdom,United States
productcategory,,,,,,
Accessories,23948.0,118127.0,48032.0,35083.0,42593.0,303515.0
Bikes,1323821.0,11636381.0,3560666.0,1543016.0,3405747.0,44832752.0
Clothing,42916.0,378948.0,128092.0,71619.0,118829.0,1037437.0
Components,203651.0,2244470.0,870748.0,334270.0,711840.0,7434097.0


### Create visualization

In [5]:
#Save the pivot table to workbook
report_table.to_excel('Sales_Report1.xlsx',
                      sheet_name='report',
                      startrow=3)

In [6]:
#Load the workbook
wb = load_workbook('Sales_Report1.xlsx')
ws = wb['report']
#
# cell references (from the spreadsheet) 
min_col = wb.active.min_column
max_col = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

In [7]:
# barchart
barchart = BarChart()
barchart.height = 10 # default is 7.5
barchart.width = 24 # default is 15
#locate data and categories
data = Reference(ws,
                 min_col=min_col+1, #including col headers
                 max_col=max_col,
                 min_row=min_row,
                 max_row=max_row) 
categories = Reference(ws,
                       min_col=min_col,
                       max_col=min_col,
                       min_row=min_row+1, #exclude row headers
                       max_row=max_row) 
# adding data and categories
barchart.add_data(data, titles_from_data=True)
barchart.set_categories(categories)
# chart location
ws.add_chart(barchart, "A11")
barchart.title = 'Sales by Product line'


In [8]:
# set the width of columns
dim_holder = DimensionHolder(worksheet=ws)

for col in range(ws.min_column, ws.max_column + 1):
    dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=20)

ws.column_dimensions = dim_holder

In [9]:
ws['A1'].value = 'Sales by Country'
ws['A2'].value = 'BI Insights Inc'
# Set title formatting
work_sheet_a1 = ws['A1']
work_sheet_a1.font = Font(size=23, underline='single', color='778899', bold=True) #italic=True
ws.merge_cells("A1:G1")
# format numbers
for r in range(5,10):
    ws[f'B{r}'].number_format ='"$"#,##0_);("$"#,##0)'
    ws[f'C{r}'].number_format ='"$"#,##0_);("$"#,##0)'
    ws[f'D{r}'].number_format ='"$"#,##0_);("$"#,##0)'
    ws[f'E{r}'].number_format ='"$"#,##0_);("$"#,##0)'
    ws[f'F{r}'].number_format ='"$"#,##0_);("$"#,##0)'
    ws[f'G{r}'].number_format ='"$"#,##0_);("$"#,##0)'

In [10]:
alphabet = list(string.ascii_uppercase)
excl_col = alphabet[0:max_col]
print(excl_col)

['A', 'B', 'C', 'D', 'E', 'F', 'G']


In [12]:
# Style for the total row
total_style = Font( bold =True)

In [13]:
# Add totals to columns B-G
thin = Side(border_style="thin", color="000000")
for i in excl_col:
    if i!='A':
        ws[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
        ws[f'{i}{max_row+1}'].number_format ='"$"#,##0_);("$"#,##0)'
        ws[f'{i}{max_row+1}'].font = total_style
# adding label
ws[f'{excl_col[0]}{max_row+1}'] = 'Total'
ws[f'A{max_row+1}'].font = Font(size=11, bold=True) #italic=True
ws[f'A{max_row+1}'].alignment = Alignment(horizontal='center')
ws[f'A{max_row+1}'].border = Border(top=thin, left=thin, right=thin, bottom=thin)

In [14]:
wb.save('Sales_Report1.xlsx')

### Convert the code to a funciton

In [ ]:
def automate_report(file_name):
    df = pd.read_csv(file_name)
    df = df[['productcategory', 'Country', 'Sales']]
    report_table = df.pivot_table(index='productcategory',
                                      columns='Country',
                                      values='Sales',
                                      aggfunc='sum').round(0)
    #
    report_table.to_excel('Sales_Report.xlsx',sheet_name='report', startrow=3)
    #
    wb = load_workbook('Sales_Report.xlsx')
    ws = wb['report']
    # cell references (original spreadsheet) 
    min_col = wb.active.min_column
    max_col = wb.active.max_column
    min_row = wb.active.min_row
    max_row = wb.active.max_row
    # barchart
    barchart = BarChart()
    barchart.height = 10 # default is 7.5
    barchart.width = 24 # default is 15
    #locate data and categories
    data = Reference(ws, min_col=min_col+1, max_col=max_col, min_row=min_row, max_row=max_row) #including headers
    categories = Reference(ws, min_col=min_col, max_col=min_col, min_row=min_row+1, max_row=max_row) #exclude headers
    # adding data and categories
    barchart.add_data(data, titles_from_data=True)
    barchart.set_categories(categories)
    # chart location
    ws.add_chart(barchart, "A11")
    barchart.title = 'Sales by Product line'
    # Set column width
    dim_holder = DimensionHolder(worksheet=ws)
    for col in range(ws.min_column, ws.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=20)
    #
    ws.column_dimensions = dim_holder
    # Set report title & subltitle
    ws['A1'].value = 'Sales by Country'
    ws['A2'].value = 'BI Insights Inc'
    work_sheet_a1 = ws['A1']
    work_sheet_a1.font = Font(size=23, underline='single', color='778899', bold=True) #italic=True
    ws.merge_cells("A1:G1")
    # Set number formatting
    for r in range(5,10):
        ws[f'B{r}'].number_format ='"$"#,##0_);("$"#,##0)'
        ws[f'C{r}'].number_format ='"$"#,##0_);("$"#,##0)'
        ws[f'D{r}'].number_format ='"$"#,##0_);("$"#,##0)'
        ws[f'E{r}'].number_format ='"$"#,##0_);("$"#,##0)'
        ws[f'F{r}'].number_format ='"$"#,##0_);("$"#,##0)'
        ws[f'G{r}'].number_format ='"$"#,##0_);("$"#,##0)'
    # Get Excel Cols
    alphabet = list(string.ascii_uppercase)
    excl_col = alphabet[0:max_column]
    # Add totals to columns B-G
    thin = Side(border_style="thin", color="000000")
    for i in excl_col:
        if i!='A':
            ws[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
            ws[f'{i}{max_row+1}'].number_format ='"$"#,##0_);("$"#,##0)'
            #ws[f'{i}{max_row+1}'].border = Border(top=thin, left=thin, right=thin, bottom=thin)
    # adding label
    ws[f'{excl_col[0]}{max_row+1}'] = 'Total'
    ws[f'A{max_row+1}'].font = Font(size=11, bold=True) #italic=True
    ws[f'A{max_row+1}'].alignment = Alignment(horizontal='center')
    ws[f'A{max_row+1}'].border = Border(top=thin, left=thin, right=thin, bottom=thin)
    # save the file
    wb.save('Sales_Report.xlsx')

In [ ]:
automate_report('data\AdvWorksData.csv')